## Status
- Data retrieved from API -> 911 matches
- Each match has 10 rows for each participant
- Attributes were preselected but need to refined further
## Next Steps
Filter Data:
- my summoner = euclidean aatrox
- my champion = Aatrox
- check number of matches left
## EDA
- analyze selected attributes
- correlation
- plots, distribution etc. (insights)
## Feature Engineering
- based on insights select final attributes to use for modeling
## From old script:
- convert time columns from seconds to minutes
- see inspiration for visualizations